# Vertex AI Memory Bank Service の機能確認

このノートブックでは、Vertex AI Agent Development Kit (ADK) を使用して、ユーザーとの会話から情報を抽出し、永続的なメモリーとして管理・検索する **Memory Bank Service** の一連のプロセスを確認します。

### 主なポイント

* **ADK セッション情報の直接記録**: ADK の `Session` オブジェクトを `VertexAiMemoryBankService` に渡すだけで、会話データをそのままメモリーバンクに登録可能です。
* **情報の自動抽出**: セッションデータから、ユーザーの属性や好みといった重要な「ファクト（事実）」を AI が自動で抽出し、メモリーとして構造化します。
* **自然言語による検索**: 「好きな食べもの」といった自然なクエリーで、関連する過去のメモリーを柔軟に取得できます。

---

### ノートブックの実行フロー

1. **環境構築と初期化**: Vertex AI SDK の初期化および必要な環境変数の設定を行います。
2. **エージェントの定義**: 会話を通じてユーザープロフィールを収集する `LlmAgent` と、それを実行する `AdkApp` を作成します。
3. **Memory Bank リソースの作成**: `gemini-2.5-flash` や多言語埋め込みモデル（`text-multilingual-embedding-002`）を指定し、メモリーを格納・検索するためのエンジンを構築します。
4. **対話データの生成**: チャットクライアントを介して、カレーの好みや個人の属性（職業・家族構成など）に関する擬似的な対話セッションを生成します。
5. **メモリーの登録**: 生成されたセッションを `add_session_to_memory` メソッドを使用して、メモリーバンクへ保存します。
6. **検索と検証**: 特定のテーマに関連するメモリーを `search_memory` メソッドで検索して、その抽出内容を確認します。

## 事前準備

In [1]:
import os
from IPython.display import HTML, Markdown, display
import vertexai
from google.adk.agents.llm_agent import LlmAgent
from google.adk.memory import VertexAiMemoryBankService
from google.genai.types import Content, Part
from vertexai.agent_engines import AdkApp

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(
    project=PROJECT_ID, location=LOCATION,
)

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

## セッション情報取得用のエージェントを作成

In [2]:
instruction = '''
Have a friendly conversation with the user.
Try to ask questions to collect user's profile.
'''

friendly_agent = LlmAgent(
    name='friendly_agent',
    model='gemini-2.5-flash',
    description='Agent to have a friendly conversation',
    instruction=instruction,
)

friendly_agent_app = AdkApp(
    agent=friendly_agent,
    app_name='friendly_agent_app',
)

In [3]:
# Chat client to test AdkApp
class ChatClient:
    def __init__(self, app, user_id='default_user'):
        self._app = app
        self._user_id = user_id
        self._session_id = None

    async def async_stream_query(self, message):
        if not self._session_id:
            session = await self._app.async_create_session(
                user_id=self._user_id,
            )
            self._session_id = getattr(session, 'id', None) or session['id']

        result = []
        async for event in self._app.async_stream_query(
            user_id=self._user_id,
            session_id=self._session_id,
            message=message,
        ):
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response:
                    result.append(response)
        return '\n'.join(result)

## Memory Bank リソースを作成

In [5]:
client = vertexai.Client(project=PROJECT_ID, location=LOCATION)

agent_engine = client.agent_engines.create(
    config={
        'display_name': 'Memory Bank Resource',
        'context_spec': {
            'memory_bank_config': {
                'generation_config': {
                    'model': f'projects/{PROJECT_ID}/locations/{LOCATION}/publishers/google/models/gemini-2.5-flash'
                },
                'similarity_search_config': {
                    # This is important to handle non-English contents
                    'embedding_model': f'projects/{PROJECT_ID}/locations/{LOCATION}/publishers/google/models/text-multilingual-embedding-002'
                },
            }
        }
    }
)

In [6]:
memory_bank_service = VertexAiMemoryBankService(
    agent_engine_id=agent_engine.api_resource.name.split('/')[-1],
    project=PROJECT_ID,
    location=LOCATION,
)

既存のメモリーを削除

In [ ]:
response = client.agent_engines.memories.list(
    name=agent_engine.api_resource.name
)

for memory in response:
    print(f'Deleting: {memory.name}')
    client.agent_engines.memories.delete(
        name=memory.name
    )

## セッションデータを作成

In [30]:
chat_client = ChatClient(friendly_agent_app)

query = '''
カレーが大好きです。
'''
response = await chat_client.async_stream_query(query)
display(Markdown(response))

こんにちは！カレー、美味しいですよね！私も大好きです！🍛✨

どんなカレーがお好きですか？
例えば、日本の家庭的なカレー、インドカレー、タイカレーとか、いろいろありますよね！😊

In [31]:
query = '''
やっぱり本格的なインドカレー。定番のチキンカレーとナンのセットがいいですね。
'''
response = await chat_client.async_stream_query(query)
display(Markdown(response))

わかります！本格的なインドカレー、美味しいですよね！特にチキンカレーとナンの組み合わせは最高です！🤤

チキンカレーは、辛口がお好きですか？それとも、マイルドでクリーミーなタイプがお好みですか？
ナンも、プレーンなものがお好きですか、それともガーリックナンやチーズナンなどもいかれますか？😊

In [32]:
query = '''
高田馬場のカレー屋によく行きます。
'''
response = await chat_client.async_stream_query(query)
display(Markdown(response))

おー、高田馬場ですか！高田馬場は「カレー激戦区」と言われるくらい、美味しいカレー屋さんがたくさんありますよね！✨

もしよろしければ、高田馬場でお気に入りのカレー屋さんがあれば、教えていただけますか？どんなところが美味しいと感じるのか、ぜひ聞かせてください！😊

## セッションデータを Memory Bank に登録

In [33]:
user_id = chat_client._user_id
session_id = chat_client._session_id
app_name = chat_client._app._tmpl_attrs['app_name'] 

session = await chat_client._app.async_get_session(
    user_id = user_id,
    session_id = session_id,
)

await memory_bank_service.add_session_to_memory(session)

記録されたメモリーを確認します。

In [34]:
# 非同期の記録が完了するまで少し待ちます。
!sleep 10

response = client.agent_engines.memories.retrieve(
    name=agent_engine.api_resource.name,
    scope={'app_name': app_name, 'user_id': user_id}
)
for item in response:
    print(f'- {item.memory.fact}')

- I often go to curry restaurants in Takadanobaba.
- I prefer authentic Indian curry, specifically a set with standard chicken curry and naan.
- I like curry.


さらにセッションデータを追加します。

In [35]:
chat_client = ChatClient(friendly_agent_app)

query = '''
以下の事実をリストアップして。

- ４０代男性
- 家族は妻と小学生の息子
- 牡羊座
- IT関連企業のエンジニア
- 趣味は散歩と読書
- 結婚記念日に家族でフレンチを食べた
'''
response = await chat_client.async_stream_query(query)
display(Markdown(response))

こんにちは！お知らせいただいた事実をリストアップしますね。

---
**ユーザーさんのプロフィール情報**

*   **年齢層:** 40代男性
*   **ご家族:** 妻と小学生の息子さん
*   **星座:** 牡羊座
*   **ご職業:** IT関連企業のエンジニア
*   **ご趣味:** 散歩と読書
*   **最近の出来事:** 結婚記念日にご家族でフレンチを召し上がった

---

色々な情報、ありがとうございます！ご家族でフレンチ、素敵ですね。
もしよろしければ、普段はどんな場所を散歩されることが多いですか？自然豊かな場所や、街中をぶらり、など…。

In [36]:
user_id = chat_client._user_id
session_id = chat_client._session_id
app_name = chat_client._app._tmpl_attrs['app_name'] 

session = await chat_client._app.async_get_session(
    user_id = user_id,
    session_id = session_id,
)

await memory_bank_service.add_session_to_memory(session)

In [37]:
# 非同期の記録が完了するまで少し待ちます。
!sleep 10

response = client.agent_engines.memories.retrieve(
    name=agent_engine.api_resource.name,
    scope={'app_name': app_name, 'user_id': user_id}
)
for item in response:
    print(f'- {item.memory.fact}')

- I often go to curry restaurants in Takadanobaba.
- I prefer authentic Indian curry, specifically a set with standard chicken curry and naan.
- I like curry.
- I ate French food with my family on our wedding anniversary.
- My family consists of my wife and elementary school son.
- My hobbies are walking and reading.
- My zodiac sign is Aries.
- I am a man in my 40s.
- I am an engineer at an IT company.


## Memory Bank に対してクエリーを実行

In [48]:
response = await memory_bank_service.search_memory(
    app_name=app_name,
    user_id=user_id,
    query='好きな食べもの',
)

for item in response.memories:
    print(f'- {item.content.parts[0].text}')

- I like curry.
- I often go to curry restaurants in Takadanobaba.
- I prefer authentic Indian curry, specifically a set with standard chicken curry and naan.


In [49]:
response = await memory_bank_service.search_memory(
    app_name=app_name,
    user_id=user_id,
    query='家族構成',
)

for item in response.memories:
    print(f'- {item.content.parts[0].text}')

- My family consists of my wife and elementary school son.
- I ate French food with my family on our wedding anniversary.
- I am a man in my 40s.
